In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
# Single family model
# made up of home and events
# start with a weekly calendar
# assign probability distributions

In [86]:
class Place():
    """ A place contains people in an event. It has a volume of air, a degree of ventilation,
    """
    def __init__(self, volume):
        """ init """
        # variables between 0 and 1
        self.case_density = 0.05 # 1 in 20 infected and transmissable

        # Variables for event space, m^3
        self.volume = volume
        self.ventilation = 0.1 # volume percent removed per hour
        self.loud = 0.5 # 1 = loud choir. 0.5 = loud cafe. 0.1 = library?
        self.mask = 0.05  # 5% mask wearing
        self.lft = 0.01 # proportion of those taking and acting on LFT
        self.starting_virus_density = 0 # particles/m^3


In [87]:
cafe = Place(volume=5*15)

In [120]:
class Crowd():
    """ Create a random crowd with given features """
    def __init__(self, size=25, case_density=0.05, lfr_density=0.1, mask_density=0.1):
        self.size = size
        self.case_density = case_density
        self.lft_density = lfr_density
        self.mask_density = mask_density
        self.peer_pressure = 0.1 # How likely will I wear a mask given most are not?

        # Calculations
        self.people_infected = np.round(self.size * self.case_density)
        self._people_infected()

    def _people_infected(self):
        """ probability of people being transmissiable.
        Sample from a bimomial distribution """
        mean_infected = self.case_density * self.size
        scenarios = np.random.binomial(self.size, self.case_density, 1000)
        self.people_infected = np.random.choice(scenarios, size=None)

    def people(self):
        return self.size
    
    def infected(self):
        return self.people_infected

In [121]:
cafe_crowd = Crowd()

In [122]:
class Me():
    def __init__(self, lfr=0.1, mask=0.9, risk_appetite=0.2):
        # Variables for me
        self.immunity = 0.1 # 1 in 10 chance of no infection even with virus particles
        self.mask_dilution = 0.25 # guess. Mask only allows 25% particles to flow

In [123]:
me = Me()

In [125]:
class Event():
    """ And event is a combination of people and place """
    def __init__(self, me, crowd, place, duration):
        """  """
        self.me = me
        self.crowd = crowd
        self.place = place
        self.duration = duration # how long in this place
        self._process()
    
    def _process(self):
        # anybody infectious?
        if self.crowd.infected() == 0:
            print("Safe", self.crowd.people())
        else:
            print("Doomed", self.crowd.people(), self.crowd.infected())
    
    def _virus_met(self):
        """ If I assume infected people are there all the time, will the
        virus reach me in enough quantity? This is extrmely finger in air
        calibration.
        
        Particles diffuse over time. Ventilation acts in the opposite direction
        and may take away more than is added.
        
        Masks then dilute any that are met.
        
        I'm going to use this study to start:
            https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2768712
            
        I am also going to assume a threshold of 300 particles for Omicron to trigger
        infection - with some binomial randomisation
        """
        pass

    def _brownian_motion(self):
        """ Model diffusion using brownian motion. Assume individuals have a fixed production
        of virus per unit time. This spreads through a volume over time increasing the density.
        But it is diluted by ventilation. """
        pass

In [126]:
# Experiment with different virus density
for _ in range(30):
    people = round(np.random.normal(15, 8))
    people = max(people, 0)
    cafe_crowd = Crowd(size=people, case_density=0.05)
    ppl = Event(me=me, place=ww_cafe, crowd=cafe_crowd, duration=120)

Safe 12
Safe 5
Safe 11
Safe 16
Doomed 32 5
Safe 4
Safe 4
Safe 8
Doomed 20 1
Safe 5
Safe 9
Doomed 10 2
Doomed 25 1
Safe 10
Safe 4
Doomed 16 1
Safe 15
Doomed 16 2
Doomed 33 2
Doomed 8 2
Safe 22
Doomed 16 1
Doomed 12 1
Doomed 16 1
Safe 12
Doomed 4 1
Safe 7
Doomed 23 2
Safe 4
Doomed 6 1


In [100]:
# Risk appetite changes the frequency of events, and those events 
# are more cautious. It can be a wrapper around calls to crowd
# and events

## Equations

Build equations to govern build up or reduction of virus

$$
    place.virus\_density = place.start\_density + event.duration(corwd.production\_density - place.ventilation\_density)
$$
  
$$
    production\_density = crowd.infected\_people * crowd.production\_rate * crowd.mask
$$
  
$$
    crowd.infected\_people = random(crowd.virus\_density * crowd.size) * crowd.lft\_density
$$

Where you see random - this is a sample from a binomial distribution: Just mulutiplying everything by the average doesn't give a good expressiuon for the variation which occurs in small settings. And infected people are integers, not floats.

When you play with the variables, the biggest risk to infection is how many infections are in the community. It is hard to calibrate the other variables, but I suspect duration contributes the most. For a short point in time, there was widespread delibarate non-symptomatic LFT - people testing before they meet others. 

It was really interesting playing with the values to see what provides for no infected people on a regular basis. And it really brings to life how big a difference the case density makes. I don't have a lot of time to research papers to calibrate the estimates I have used. The intent of this work is to understand how the variables interact.

